<a href="https://colab.research.google.com/github/user292606/InsightCart/blob/main/Inventory_and_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import Counter
from collections import defaultdict
from google.colab import files
uploaded = files.upload()

Saving sales_df.csv to sales_df.csv


In [ ]:
import pandas as pd
sales = pd.read_csv('sales_df.csv')
sales.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [ ]:
sales=sales.drop(columns=['transaction_id', 'staff_id', 'customer_id', 'instore_yn', 'line_item_id', 'line_item_amount'])
sales

,transaction_date,transaction_time,sales_outlet_id,order,product_id,quantity,unit_price,promo_item_yn
0,2019-04-01,12:04:43,3,1,52,1,2.50,N
1,2019-04-01,15:54:39,3,1,27,2,3.50,N
2,2019-04-01,14:34:59,3,1,46,2,2.50,N
3,2019-04-01,16:06:04,3,1,23,2,2.50,N
4,2019-04-01,19:18:37,3,1,34,1,2.45,N
...,...,...,...,...,...,...,...,...
49889,2019-04-29,16:51:58,8,1,30,1,3.00,N
49890,2019-04-29,16:51:14,8,1,25,2,2.20,N
49891,2019-04-29,11:17:36,8,1,31,1,2.20,N
49892,2019-04-29,15:45:52,8,1,44,2,2.50,N


In [ ]:
sales=sales[sales['sales_outlet_id']==8]
sales=pd.DataFrame(sales)
sales

,transaction_date,transaction_time,sales_outlet_id,order,product_id,quantity,unit_price,promo_item_yn
1102,2019-04-01,09:05:11,8,1,42,1,2.50,N
1103,2019-04-01,10:38:16,8,1,57,1,3.10,N
1104,2019-04-01,10:38:16,8,1,69,1,3.25,N
1105,2019-04-01,08:57:46,8,1,54,1,2.50,N
1106,2019-04-01,09:21:13,8,1,54,2,2.50,N
...,...,...,...,...,...,...,...,...
49889,2019-04-29,16:51:58,8,1,30,1,3.00,N
49890,2019-04-29,16:51:14,8,1,25,2,2.20,N
49891,2019-04-29,11:17:36,8,1,31,1,2.20,N
49892,2019-04-29,15:45:52,8,1,44,2,2.50,N


In [ ]:
from datetime import datetime
class Ingredient:
    def __init__(self, name, quantity=0, warning_threshold=5000):
        self.name = name
        self.quantity = quantity
        self.warning_threshold = warning_threshold

    def add_quantity(self, amount):
        self.quantity += amount
        print(f"Added {amount} {self.name} to inventory.")

    def reduce_quantity(self, amount):
        if self.quantity >= amount:
            self.quantity -= amount
            #print(f"Reduced {amount} {self.name} from inventory.")
        else:
            print(f"Insufficient {self.name} in inventory.")

    def check_warning(self):
        if self.quantity <= self.warning_threshold:
            print(f"Warning: {self.name} quantity is low ({self.quantity}).")


In [ ]:
class MenuItem:
    def __init__(self, name, price, ingredients):
        self.name = name
        self.price = price
        self.ingredients = ingredients

class Sale:
    def __init__(self, item_name, quantity, revenue):
        self.item_name = item_name
        self.quantity = quantity
        self.revenue = revenue
        self.transaction_time = None

In [ ]:
class InventorySystem:
    def __init__(self):
        self.ingredients = {}
        self.menu = {}
        self.sales = []
        self.product_sales = defaultdict(int)
        self.product_revenue = defaultdict(float)
        self.total_sales = 0.0
        self.busiest_orders = Counter()

    def add_ingredient(self, name, quantity):
        if name in self.ingredients:
            self.ingredients[name].add_quantity(quantity)
        else:
            self.ingredients[name] = Ingredient(name, quantity)
            #print(f"Added {name} to inventory with {quantity} quantity.")

    def add_menu_item(self, name, price, ingredients):
        self.menu[name] = MenuItem(name, price, ingredients)
        #print(f"Added '{name}' to the menu.")

    def record_sale_with_time(self, item_name, quantity, transaction_time):
        if item_name in self.menu:
            menu_item = self.menu[item_name]
            # Convert the price to float to handle potential string values
            revenue = float(menu_item.price.replace(',', '.')) * quantity
            self.sales.append(Sale(item_name, quantity, revenue))
            self.product_sales[item_name] += quantity
            self.product_revenue[item_name] += revenue
            self.total_sales += revenue
            self.update_busiest_orders(transaction_time) # Pass transaction_time to the method
            for ingredient, qty_needed in menu_item.ingredients.items():
                self.ingredients[ingredient].reduce_quantity(qty_needed * quantity)
        #    print(f"Recorded sale of {quantity} {item_name}.")
        #else:
        #    print(f"{item_name} is not available in the menu.")

    def update_busiest_orders(self, transaction_time):
        self.busiest_orders[transaction_time] += 1

    def load_ingredients_from_file(self, filename):
        df = pd.read_csv(filename, sep=';')
        for index, row in df.iterrows():
            self.add_ingredient(row['Name'], row['Quantity'])

    def load_menu_from_file(self, filename):
        df = pd.read_csv(filename, sep=';')
        for index, row in df.iterrows():
            ingredients = {}
            for item in row['Ingredients'].split(','):
                ing_name, ing_quantity = item.split(':')
                ingredients[ing_name.strip()] = float(ing_quantity)
            self.add_menu_item(row['Name of Product'], row['Price'], ingredients)

    def display_inventory(self):
        print("Current Inventory:")
        for ingredient_name, ingredient in self.ingredients.items():
            print(f"{ingredient_name}: {ingredient.quantity}")
            ingredient.check_warning()

    def display_menu(self):
        print("Menu:")
        for item_name, item in self.menu.items():
            print(f"{item_name}: ${item.price}")
            print("Ingredients:")
            for ingredient, quantity in item.ingredients.items():
                print(f"- {ingredient}: {quantity}")

    def display_sales_summary(self):
        print("\nSales Summary:")
        print(f"Total Sales: ${self.total_sales:.2f}")

        # Product sales and revenue
        print("\nProduct Sales (Quantity Sold):")
        sorted_products_by_sales = sorted(self.product_sales.items(), key=lambda x: x[1], reverse=True)
        for product, quantity in sorted_products_by_sales:
            revenue = self.product_revenue[product]
            print(f"{product}: {quantity} units, Revenue: ${revenue:.2f}")

        sorted_product_revenue = sorted(self.product_revenue.items(), key=lambda x: x[1], reverse=True)
        print("\nProduct Revenue (Sorted by Revenue):")
        for product, revenue in sorted_product_revenue:
            quantity = self.product_sales[product]
            print(f"{product}: {quantity} units, Revenue: ${revenue:.2f}")

        #busiest time
        busiest_times = self.busiest_orders.most_common(3)  #top 3 busiest times
        print("\nBusiest Order Times:")
        for time, quantity in busiest_times:
            formatted_time = datetime.strptime(time, "%H:%M:%S").strftime("%H:%M:%S")
            #interval grouping
            hour = datetime.strptime(time, "%H:%M:%S").hour
            interval_quantity = 0
            for t, qty in self.busiest_orders.items():
                order_hour = datetime.strptime(t, "%H:%M:%S").hour
                if order_hour == hour:
                    interval_quantity += qty

            start_time = f"{hour:02}:00:00"
            end_time = f"{hour:02}:59:59"
            print(f"{start_time}-{end_time}: {interval_quantity} units")

    def record_sale_with_time(self, item_name, quantity, transaction_time):
        if item_name in self.menu:
            menu_item = self.menu[item_name]
            #converting the price to float to handle potential string values
            revenue = float(menu_item.price.replace(',', '.')) * quantity
            self.sales.append(Sale(item_name, quantity, revenue))
            self.product_sales[item_name] += quantity
            self.product_revenue[item_name] += revenue
            self.total_sales += revenue
            self.update_busiest_orders(transaction_time)
            for ingredient, qty_needed in menu_item.ingredients.items():
                self.ingredients[ingredient].reduce_quantity(qty_needed * quantity)
            #print(f"Recorded sale of {quantity} {item_name}.")
        else:
            print(f"{item_name} is not available in the menu.")



In [ ]:
def main():
    inventory = InventorySystem()

    #Load initial data from CSV files
    inventory.load_ingredients_from_file('Ingredients.csv')
    inventory.load_menu_from_file('Menu.csv')

    #Load sales data
    sales_data = sales
    sales_data = sales_data[sales_data['transaction_date']=='2019-04-01']

    # Load menu data
    mnu_data = pd.read_csv('Menu.csv', sep=';')

    #mapping dictionary from product_id to menu item name
    product_to_menu = {}

    #Populate the product_to_menu dictionary based on menu_data
    for index, row in menu_data.iterrows():
        product_to_menu[row['Product ID']] = row['Name of Product']

    #Simulate sales based on sales_data
    for index, row in sales_data.iterrows():
        product_id = row['product_id']
        quantity = row['quantity']
        if product_id in product_to_menu:
            menu_item = product_to_menu[product_id]
            inventory.record_sale_with_time(menu_item, quantity, row['transaction_time'])
        else:
            print(f"Product ID {product_id} does not map to a menu item.")


    #updated inventory status and sales history
    print("\nUpdated Inventory Status:")
    inventory.display_inventory()
    inventory.display_sales_summary()

if __name__ == "__main__":
    main()


Product ID 77 does not map to a menu item.
Product ID 79 does not map to a menu item.
Product ID 79 does not map to a menu item.
Product ID 76 does not map to a menu item.
Product ID 79 does not map to a menu item.
Product ID 76 does not map to a menu item.
Product ID 74 does not map to a menu item.
Product ID 79 does not map to a menu item.
Product ID 71 does not map to a menu item.
Product ID 71 does not map to a menu item.
Product ID 69 does not map to a menu item.
Insufficient Susu Dairy in inventory.
Product ID 77 does not map to a menu item.
Product ID 70 does not map to a menu item.
Insufficient Susu Dairy in inventory.
Product ID 77 does not map to a menu item.
Insufficient Susu Dairy in inventory.
Insufficient Susu Dairy in inventory.
Insufficient Susu Dairy in inventory.
Insufficient Susu Dairy in inventory.
Insufficient Susu Dairy in inventory.
Insufficient Susu Dairy in inventory.
Insufficient Susu Dairy in inventory.
Product ID 74 does not map to a menu item.
Insufficient 